## Creating the Connection (via Tier 1)

Sqlite is part of the python standard library 

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

To connect to a database use the sqlite3.connect(). You can pass in a file or create a database from memory. In this case we wrapped the connect in a try-except just in case it doesn't load.

In [2]:
try: 
    conn = sqlite3.connect("sqlite_db_pythonsqlite.db")
except Error as e:
    print(e)


conn is a connection object with many associated methods and attributes. The cursor instance has an execute method that allows us to execute sql statements. After executing the fetchall() returns all the rows of a query result as a list of tuples. There are other fetches such as fetchone() which fetches one row. And fetchmany(n) returns the number of rows specified by n. When called repeatedly this method fetches the next set of rows of a query result.

*Note that conn has an execute method that creates an intermediate cursor object by calling the cursor method, then calls the cursor's execute method. However, it seems that the standard protocol is to create the cursor separately*

**It is good practice to close the connection after you are done which is why it is wrapped in a with statement that takes care of that for us**

In [3]:
with conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM FACILITIES")
    rows = cur.fetchall()

In [25]:
for row in rows:
    print(row)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [5]:
_ = pd.read_sql_query("SELECT * FROM Facilities", conn)
_

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

To figure out revenue I need to group the bookings by facid. Once grouped I need to calculate the revenue for each individual booking. To do this I need to know: if it was a guest or memeber, the guest or member cost, and the number of slots. Then I need to sum up all those calculations for each group. Filter the groups by revenue less than 1000 and then order them. 

In [26]:
q10_query = """
        SELECT name AS Facility, SUM(price) AS Revenue
          FROM 
               (SELECT bookid, name, memid,
                       CASE 
                           WHEN memid > 0 THEN membercost * slots
                           ELSE guestcost * slots 
                           END AS price	
 
                  FROM Bookings
                       INNER JOIN Facilities
                       USING (facid)
                       
                ) AS booking_price

        GROUP BY name
       HAVING revenue < 1000
        ORDER BY revenue DESC
        """
q10 = pd.read_sql_query(q10_query, conn)
q10

,Facility,Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [27]:
q11_query = """
        SELECT m1.surname, m1.firstname, 
               COALESCE(m2.surname, ' '), COALESCE(m2.firstname, ' ')
          FROM Members as m1
               LEFT JOIN (SELECT * FROM Members WHERE memid > 0) as m2
               ON m1.recommendedby = m2.memid

         WHERE m1.memid > 0
         ORDER BY m1.surname
         """
q11 = pd.read_sql_query(q11_query, conn)
q11

,surname,firstname,"COALESCE(m2.surname, ' ')","COALESCE(m2.firstname, ' ')"
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,Jemima,,
9,Farrell,David,,


*Note: I couldn't concat the first and last names into one column. They kept return 0s when I tried.*

### Q12: Find the facilities with their usage by member, but not guests### 

In [30]:
q14_query = """
        SELECT fullname AS 'Member', name, 
               SUM(slots) AS 'Usage'
          FROM 
               (SELECT (surname || ' '|| firstname) AS fullname, name, slots
                  FROM Bookings
                       LEFT JOIN Facilities
                       USING (facid)
 
                       INNER JOIN (SELECT * FROM Members WHERE memid > 0) AS members_only
                       USING (memid)
                       
               ) AS member_data
 
         GROUP BY fullname, name
         """
q14 = pd.read_sql_query(q14_query, conn)
q14

,Member,name,Usage
0,Bader Florence,Badminton Court,27
1,Bader Florence,Massage Room 2,4
2,Bader Florence,Pool Table,23
3,Bader Florence,Snooker Table,66
4,Bader Florence,Squash Court,4
...,...,...,...
197,Tupperware Hyacinth,Squash Court,2
198,Worthington-Smyth Henry,Badminton Court,15
199,Worthington-Smyth Henry,Massage Room 1,2
200,Worthington-Smyth Henry,Pool Table,37


### Q13: Find the facilities usage by month, but not guests

For this query I need to sum up the number of times a facility was used (slots) for each individual facility. Excluding members. partitioning by month.

In [29]:
q13_query = """
        SELECT name AS 'Facility', Month, 
               SUM(slots) AS 'Usage'
          FROM 
               (SELECT name, slots, strftime('%m', starttime) AS Month
                  FROM Bookings
                       LEFT JOIN Facilities
                       USING (facid)
                 
                 WHERE memid > 0
               ) as month_extract
               
         GROUP BY name, Month
         """
q13 = pd.read_sql_query(q13_query, conn)
q13

,Facility,Month,Usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110


*Note the original query had MONTHNAME(starttime) AS Month instead of strftime, however sqlite doesn't have a monthname function